In [1]:
from fileinput import filename
import pandas as pd
from datetime import datetime, timedelta
import pytz
import email
import io
from google.oauth2 import service_account
from utils import unzip_csv , send_email, get_sheet
import csv
from urllib.request import urlopen


report_sheet, report_df = get_sheet(spreadsheet='PLanalysis', sheet_num=0)
data_sheet, data_df = get_sheet(spreadsheet='PLanalysis', sheet_num=-1)

In [50]:
data_df.dtypes

Date                object
Company             object
Amount              object
Exchange            object
Segment             object
Scrip Code          object
Instrument Type     object
Strike Price        object
Expiry              object
Trade Num           object
Trade Time          object
Side                object
Quantity           float64
Price               object
dtype: object

In [51]:
# cleaning
data_df['Quantity'] = data_df['Quantity'].astype(int)
data_df['Date'] = pd.to_datetime(data_df['Date'], format='%d-%m-%Y')
data_df['Trade Time'] = pd.to_datetime(data_df['Trade Time'], format='%H:%M:%S')
data_df['Amount'] = data_df['Amount'].replace('[\₹,]', '', regex=True).astype(float)
data_df['Price'] = data_df['Price'].replace('[\₹,]', '', regex=True).astype(float)

In [52]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             123 non-null    datetime64[ns]
 1   Company          123 non-null    object        
 2   Amount           123 non-null    float64       
 3   Exchange         123 non-null    object        
 4   Segment          123 non-null    object        
 5   Scrip Code       123 non-null    object        
 6   Instrument Type  123 non-null    object        
 7   Strike Price     123 non-null    object        
 8   Expiry           123 non-null    object        
 9   Trade Num        123 non-null    object        
 10  Trade Time       93 non-null     datetime64[ns]
 11  Side             123 non-null    object        
 12  Quantity         123 non-null    int64         
 13  Price            123 non-null    float64       
dtypes: datetime64[ns](2), float64(2), int64(1)

In [158]:
end_date = datetime.now()
start_date = end_date - timedelta(days=30)
print(start_date.date(), end_date.date())
result_df = data_df[(data_df['Date'] >= start_date) & (data_df['Date'] <= end_date)].copy()
result_df = result_df[result_df.Segment=='EQ'].groupby(['Company', 'Side', 'Date']).agg({'Amount': 'sum', 'Quantity': 'sum', 'Price': 'mean'})

result_df = result_df.reset_index()
result_df = result_df.sort_values(by=['Company', 'Date'], ascending=[True, False])
result_df

2023-12-12 2024-01-11


,Company,Side,Date,Amount,Quantity,Price
0,APOLLO TYRE.,Buy,2023-12-29,14272.00,32,446.000000
1,Aditya Birla Capital Ltd,Buy,2023-12-26,19267.15,120,160.551316
2,Anmol India Limited,Buy,2023-12-22,565.65,9,62.850000
3,BHARAT FORGE,Sell,2024-01-04,6260.00,5,1252.000000
5,CENTRAL DEPO SER (I) LTD,Sell,2024-01-05,22632.00,12,1886.000000
4,CENTRAL DEPO SER (I) LTD,Buy,2023-12-20,3700.00,2,1850.000000
6,DEEPAK FERT,Buy,2024-01-03,19320.00,28,690.000000
7,Delhivery Limited,Buy,2024-01-08,20107.50,50,402.150000
8,HDFC BANK LT,Sell,2023-12-27,15300.00,9,1700.000000
9,Indian Railway Finance,Buy,2024-01-02,19075.20,192,99.350000


In [159]:
# oldest order for each company
min_dates = result_df.groupby('Company')['Date'].idxmin()
result_df = result_df.loc[~(result_df.index.isin(min_dates) & (result_df['Side']=='Sell'))]
# oldest order for each company
max_dates = result_df.groupby('Company')['Date'].idxmax()
result_df = result_df.loc[~(result_df.index.isin(max_dates) & (result_df['Side']=='Buy'))]

In [162]:
# companies with single record/row
single_record = result_df.groupby('Company').size()[result_df.groupby('Company').size() < 2].index.to_list()
result_df = result_df[~result_df.Company.isin(single_record)]

In [164]:
# companies with first sell order quantity greater than first buy order quantity
temp_df = result_df.groupby(['Company','Side'])['Quantity'].min().unstack()
result_df = result_df[~result_df.Company.isin(temp_df[temp_df['Buy'] < temp_df['Sell']].index.to_list())]

In [168]:
result_df['Revenue'] = result_df.apply(lambda x: -x['Amount'] if x['Side']=='Buy' else x['Amount'], axis=1)
result_df

,Company,Side,Date,Amount,Quantity,Price,Revenue
14,MOTH SUMI SY,Sell,2024-01-10,1091.0,10,109.1,1091.0
13,MOTH SUMI SY,Buy,2023-12-29,1019.0,10,101.9,-1019.0
18,SUMITOMO CHEMICAL INDIA LIMITE,Sell,2024-01-08,20825.0,49,425.0,20825.0
17,SUMITOMO CHEMICAL INDIA LIMITE,Buy,2024-01-03,19869.5,49,405.5,-19869.5
20,TATA CHEMICALS LTD.,Sell,2024-01-01,10206.0,9,1134.0,10206.0
19,TATA CHEMICALS LTD.,Buy,2023-12-22,9162.0,9,1018.0,-9162.0


In [169]:
result_df.Revenue.sum()

2071.5